In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats

df_2004 = pd.read_stata('usu_individual_T104.dta', convert_categoricals=False)
df_2024 = pd.read_excel('usu_individual_T124.xlsx')

df_2004.columns = df_2004.columns.str.upper()
df_2024.columns = df_2024.columns.str.upper()
columnas_comunes = df_2004.columns.intersection(df_2024.columns)

df_2004_filtrado = df_2004[columnas_comunes]
df_2024_filtrado = df_2024[columnas_comunes]
df_combinado = pd.concat([df_2004_filtrado, df_2024_filtrado], ignore_index=True)

df_eph_nea = df_combinado[df_combinado['REGION'] == 41].copy()

# Variable EDAD2
if 'CH06' in df_eph_nea.columns:
    df_eph_nea['EDAD2'] = np.square(df_eph_nea['CH06'])
    print("Variable 'edad2' añadida exitosamente.")
else:
    print("No se encontró la columna 'CH06'.")

# Variable EDUC
def calcular_educ(row):
    if row['CH10'] == 1:
        if row['CH14'] in [98, 99]:
            return 6
        if row['CH12'] == 1:
            return 0
        elif row['CH12'] == 2:
            return row['CH14']
        elif row['CH12'] == 3:
            return row['CH14']
        elif row['CH12'] == 4:
            return 6 + row['CH14']
        elif row['CH12'] == 5:
            return 9 + row['CH14']
        elif row['CH12'] == 6:
            return 12 + row['CH14']
        elif row['CH12'] == 7:
            return 12 + row['CH14']
        elif row['CH12'] == 8:
            return 17 + row['CH14']
        elif row['CH12'] == 9:
            return row['CH14']
        else:
            return 0
    elif row['CH10'] == 2:
        if row['CH13'] == 1:
            if row['CH14'] in [98, 99]:
                return 6
            elif row['CH12'] == 1:
                return 0
            elif row['CH12'] == 2:
                return 6
            elif row['CH12'] == 3:
                return 9
            elif row['CH12'] == 4:
                return 12
            elif row['CH12'] == 5:
                return 12
            elif row['CH12'] == 6:
                return 15
            elif row['CH12'] == 7:
                return 17
            elif row['CH12'] == 8:
                return 18
            elif row['CH12'] == 9:
                return 6
            else:
                return 0
        elif row['CH13'] == 2:
            if row['CH14'] in [98, 99]:
                return 6
            elif row['CH12'] == 1:
                return 0
            elif row['CH12'] == 2:
                return row['CH14']
            elif row['CH12'] == 3:
                return row['CH14']
            elif row['CH12'] == 4:
                return 6 + row['CH14']
            elif row['CH12'] == 5:
                return 9 + row['CH14']
            elif row['CH12'] == 6:
                return 12 + row['CH14']
            elif row['CH12'] == 7:
                return 12 + row['CH14']
            elif row['CH12'] == 8:
                return 17 + row['CH14']
            elif row['CH12'] == 9:
                return row['CH14']
            else:
                return 0
    elif row['CH10'] == 3:
        return 0
    else:
        return 0

for col in ['CH10', 'CH12', 'CH13', 'CH14']:
    if col in df_eph_nea.columns:
        df_eph_nea[col] = pd.to_numeric(df_eph_nea[col], errors='coerce')

df_eph_nea['EDUC'] = df_eph_nea.apply(calcular_educ, axis=1)
print("Variable 'educ' añadida exitosamente.")

# Variable SALARIO_SEMANAL
SMVM2004 = 350
SMVM2024 = 202800
pp = round(SMVM2024 / SMVM2004, 2)

if 'ANO4' in df_eph_nea.columns and 'ESTADO' in df_eph_nea.columns and 'P21' in df_eph_nea.columns:
    condiciones_2004 = (df_eph_nea['ANO4'] == 2004) & (df_eph_nea['ESTADO'] == 1) & (df_eph_nea['P21'] > 0)
    condiciones_2024 = (df_eph_nea['ANO4'] == 2024) & (df_eph_nea['ESTADO'] == 1) & (df_eph_nea['P21'] > 0)

    df_eph_nea.loc[condiciones_2004, 'SALARIO_SEMANAL'] = ((df_eph_nea.loc[condiciones_2004, 'P21'] * pp) / 21.65) * 5
    df_eph_nea.loc[condiciones_2024, 'SALARIO_SEMANAL'] = (df_eph_nea.loc[condiciones_2024, 'P21'] / 21.65) * 5
    print("Variable 'salario_semanal' añadida exitosamente.")
else:
    print("Faltan columnas necesarias para calcular 'salario_semanal'.")

# Variable HORASTRAB
variables_a_limpiar = ['PP3E_TOT', 'PP3F_TOT']
for var in variables_a_limpiar:
    if var in df_eph_nea.columns:
        df_eph_nea[var] = df_eph_nea[var].replace([99, 999, 9999], np.nan)

if all(col in df_eph_nea.columns for col in variables_a_limpiar):
    df_eph_nea['HORASTRAB'] = df_eph_nea['PP3E_TOT'] + df_eph_nea['PP3F_TOT']
    print("Variable 'horastrab' añadida exitosamente.")
else:
    print("Faltan columnas necesarias para calcular 'horastrab'.")

df_eph_nea.to_excel('EPH_NEA_2004_2024.xlsx', index=False)
print("Archivo final guardado con todas las variables añadidas.")

respondieron = df_eph_nea[df_eph_nea['ESTADO'].notna() & (df_eph_nea['ESTADO'] != 0)]
norespondieron = df_eph_nea[(df_eph_nea['ESTADO'].isna()) | (df_eph_nea['ESTADO'] == 0)]

respondieron.to_excel('respondieron.xlsx', index=False)
norespondieron.to_excel('norespondieron.xlsx', index=False)

Variable 'edad2' añadida exitosamente.
Variable 'educ' añadida exitosamente.
Variable 'salario_semanal' añadida exitosamente.
Variable 'horastrab' añadida exitosamente.
Archivo final guardado con todas las variables añadidas.


In [3]:
# Dimensión de la base
print("Dimensión del dataframe:", respondieron.shape)

# Variables e información
print(respondieron.info())

respondieron.head()
respondieron.tail()

Dimensión del dataframe: (10182, 176)
<class 'pandas.core.frame.DataFrame'>
Index: 10182 entries, 17040 to 91157
Columns: 176 entries, CODUSU to HORASTRAB
dtypes: float64(142), object(34)
memory usage: 13.7+ MB
None


,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,...,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,EDAD2,EDUC,SALARIO_SEMANAL,HORASTRAB
91153,TQRMNOSWRHKOLOCDEGIBJ00861592,1.0,1.0,1.0,2024.0,1.0,41.0,N,12.0,368.0,...,3,3.0,4,NaN,3.0,3,2704.0,12.0,71593.533487,0.0
91154,TQRMNOSWRHKOLOCDEGIBJ00861592,1.0,2.0,1.0,2024.0,1.0,41.0,N,12.0,368.0,...,3,3.0,4,NaN,3.0,3,1764.0,6.0,NaN,NaN
91155,TQRMNOSWRHKOLOCDEGIBJ00861592,1.0,3.0,1.0,2024.0,1.0,41.0,N,12.0,368.0,...,3,3.0,4,NaN,3.0,3,361.0,11.0,NaN,NaN
91156,TQRMNOSWRHKOLOCDEGIBJ00861592,1.0,4.0,1.0,2024.0,1.0,41.0,N,12.0,368.0,...,3,3.0,4,NaN,3.0,3,256.0,10.0,NaN,NaN
91157,TQRMNOSWRHKOLOCDEGIBJ00861592,1.0,5.0,0.0,2024.0,1.0,41.0,N,12.0,368.0,...,3,3.0,4,NaN,3.0,3,16.0,0.0,NaN,NaN


In [4]:
# Hay duplicados?
print("Duplicados:", respondieron.duplicated().sum())

# Hay valores faltantes?
print("\n Missings:\n", respondieron.isnull().sum()) # conteo


Duplicados: 0

 Missings:
 CODUSU                0
NRO_HOGAR             0
COMPONENTE            0
H15                   0
ANO4                  0
                   ... 
ADECCFR               0
EDAD2                 0
EDUC                  1
SALARIO_SEMANAL    6715
HORASTRAB          2733
Length: 176, dtype: int64


In [21]:
respondieron['P21'] = pd.to_numeric(respondieron['P21'], errors='coerce')
respondieron = respondieron[respondieron['P21'] >= 0]


C:\Users\andre\AppData\Local\Temp\ipykernel_15804\2575122037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron['P21'] = pd.to_numeric(respondieron['P21'], errors='coerce')


In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy import stats

# Variables relevantes
variables = ['CH06', 'EDAD2', 'EDUC', 'P21', 'SALARIO_SEMANAL', 'HORASTRAB', 'ESTADO', 'CH04', 'PP04A', 'PP03D']
columnas = ['constante'] + variables

# Listas para conjuntos combinados
train_list = []
test_list = []

for anio in [2004, 2024]:
    df_anio = respondieron[respondieron['ANO4'] == anio]
    y = (df_anio['ESTADO'] == 2).astype(int)  # Variable target

    X_df = df_anio[variables].copy()
    X_df.insert(0, 'constante', 1)

    data = pd.concat([X_df, y.rename('target')], axis=1).dropna(subset=['target'])

    train, test = train_test_split(data, test_size=0.3, random_state=444)

    # Guardar para conjuntos combinados
    train_list.append(train)
    test_list.append(test)

    # Análisis de diferencia de medias
    tabla_dif = pd.DataFrame(index=columnas)

    for var in columnas:
        train_valid = train[[var]].dropna()
        test_valid = test[[var]].dropna()

        tabla_dif.loc[var, 'N train'] = train_valid.shape[0]
        tabla_dif.loc[var, 'Mean train'] = train_valid[var].mean()
        tabla_dif.loc[var, 'sd train'] = train_valid[var].std()

        tabla_dif.loc[var, 'N test'] = test_valid.shape[0]
        tabla_dif.loc[var, 'Mean test'] = test_valid[var].mean()
        tabla_dif.loc[var, 'sd test'] = test_valid[var].std()

        if len(train_valid[var]) > 1 and len(test_valid[var]) > 1:
            t_test = stats.ttest_ind(train_valid[var], test_valid[var], equal_var=False, nan_policy='omit')
            tabla_dif.loc[var, 't-test'] = t_test.statistic
            tabla_dif.loc[var, 'p-value'] = t_test.pvalue
        else:
            tabla_dif.loc[var, 't-test'] = None
            tabla_dif.loc[var, 'p-value'] = None

    columnas_a_redondear = ['Mean train', 'Mean test', 'sd train', 'sd test', 't-test', 'p-value']
    tabla_dif[columnas_a_redondear] = tabla_dif[columnas_a_redondear].round(2)

    tabla_dif.to_excel(f'Tabla2_diferencia_de_medias_{anio}.xlsx')
    print(f"Exportado tabla de diferencia de medias para el año {anio}")

# Guardar conjuntos combinados
train_combined = pd.concat(train_list).reset_index(drop=True)
test_combined = pd.concat(test_list).reset_index(drop=True)

train_combined.to_excel("train_combinado.xlsx", index=False)
test_combined.to_excel("test_combinado.xlsx", index=False)
print("Conjuntos combinados guardados exitosamente.")


C:\Users\andre\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Exportado tabla de diferencia de medias para el año 2004
Exportado tabla de diferencia de medias para el año 2024


C:\Users\andre\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Conjuntos combinados guardados exitosamente.


In [19]:
import statsmodels.api as sm

ocupados = train_combined[
    (train_combined['ESTADO'] == 1) &
    (train_combined['SALARIO_SEMANAL'].notnull()) &
    (train_combined['CH04'].notnull()) &  
    (train_combined['CH06'].notnull()) &  
    (train_combined['EDAD2'].notnull()) &
    (train_combined['EDUC'].notnull()) &
    (train_combined['PP04A'].notnull()) &
    (train_combined['PP03D'].notnull())
].copy()

ocupados['mujer'] = (ocupados['CH04'] == 2).astype(int)

modelos = {
    'Modelo 1': ['CH06'],
    'Modelo 2': ['CH06', 'EDAD2'],
    'Modelo 3': ['CH06', 'EDAD2', 'EDUC'],
    'Modelo 4': ['CH06', 'EDAD2', 'EDUC', 'mujer'],
    'Modelo 5': ['CH06', 'EDAD2', 'EDUC', 'mujer', 'PP04A', 'PP03D']
}

resultados = {}

for nombre, vars_indep in modelos.items():
    X = ocupados[vars_indep]
    X = sm.add_constant(X)
    y = ocupados['SALARIO_SEMANAL']
    
    modelo = sm.OLS(y, X).fit()
    resultados[nombre] = modelo

tabla_resultados = pd.DataFrame()

for nombre, modelo in resultados.items():
    resumen = pd.DataFrame({
        nombre: [
            f"{coef:.3f}" + (
                "***" if pval < 0.001 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
            ) + f"\n({se:.2f})"
            for coef, se, pval in zip(modelo.params, modelo.bse, modelo.pvalues)
        ]
    }, index=modelo.params.index)

    resumen.loc['R2'] = f"{modelo.rsquared:.3f}"
    resumen.loc['N (obs)'] = int(modelo.nobs)
    tabla_resultados = pd.concat([tabla_resultados, resumen], axis=1)

print("Tabla 2\n")
print(tabla_resultados)


Tabla 2

                        Modelo 1                 Modelo 2  \
const    31058.229***\n(3328.89)  -20470.889**\n(9179.92)   
CH06         550.485***\n(79.90)    3258.043***\n(456.94)   
R2                         0.019                    0.033   
N (obs)                     2459                     2459   
EDAD2                        NaN       -32.318***\n(5.37)   
EDUC                         NaN                      NaN   
mujer                        NaN                      NaN   
PP04A                        NaN                      NaN   
PP03D                        NaN                      NaN   

                         Modelo 3                  Modelo 4  \
const    -48339.989***\n(8922.00)  -45777.140***\n(8728.68)   
CH06        2513.266***\n(437.90)     2617.555***\n(428.36)   
R2                          0.123                     0.161   
N (obs)                      2459                      2459   
EDAD2          -21.647***\n(5.16)        -22.608***\n(5.05)   
ED